In [1]:
import os
import pandas as pd
import numpy as np

## 1. Set directories

In [2]:
output_dir_full = os.path.join(r"C:\VY-Projects\sandag\Outputs")
output_dir_sample = os.path.join(r"C:\VY-Projects\sandag\Outputs10percent")

## 2. Load trip data of original full model run

In [3]:
#load trip data - full sample
ind_trips_full = pd.read_csv(os.path.join(output_dir_full, 'indivTripData_3.csv.gz'))
jnt_trips_full = pd.read_csv(os.path.join(output_dir_full, 'jointTripData_3.csv.gz'))

## 3. Load trip data of RSM

In [4]:
ind_trips_rsm = pd.read_csv(os.path.join(output_dir_sample, 'indivTripData_1.csv'))
jnt_trips_rsm = pd.read_csv(os.path.join(output_dir_sample, 'jointTripData_1.csv'))

## Function definitions

In [7]:
# function to convert to final trip roster
def convert_to_trips(indiv_trips, joint_trips):
    joint_trips = joint_trips[['hh_id','tour_id','num_participants','trip_mode']]
    joint_trips = joint_trips.reindex(joint_trips.index.repeat(joint_trips.num_participants)).reset_index(drop=True)
    joint_trips = joint_trips.drop(columns = ['num_participants'])
    
    indiv_trips = indiv_trips[['hh_id', 'tour_id','trip_mode']]
    trips = pd.concat([joint_trips, indiv_trips], ignore_index=True).reset_index(drop=True)
    
    return trips

In [33]:
#convert to rsm trips 
trips_rsm =  convert_to_trips(ind_trips_rsm, jnt_trips_rsm)
trips_rsm.shape
trips_rsm['flag'] = 1

In [21]:
trips_org_full =  convert_to_trips(ind_trips_full, jnt_trips_full)
trips_org_full.shape

(13639617, 3)

## Create Trip roster using RSM trips and original model trips

In [42]:
# get list of all hhids in trips produced by RSM
hh_ids_rsm = list(trips_rsm['hh_id'])

In [23]:
# remove orginal model trips made by households chosen in RSM trips  
trips_org_full_unsample_hhid = trips_full.loc[~trips_full['hh_id'].isin(hh_ids_rsm)]

In [24]:
# concatenate trips from rsm and orginal model
final_trips_rsm = pd.concat([trips_rsm, trips_full_unsample_hhid], ignore_index =True).reset_index(drop=True)

In [25]:
final_trips_rsm.shape

(13585175, 3)

## Get percentage change in trips for each home zone

In [26]:
# extract trips made by houselods in RSM and Original model
trips_org_full_sample_hhid = trips_full.loc[trips_full['hh_id'].isin(hh_ids_rsm)]
trips_org_full_sample_hhid['flag'] = 1

C:\Users\vyadav\AppData\Local\Temp\ipykernel_12080\2872501130.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_org_full_sample_hhid['flag'] = 1


In [55]:
#aggregating trips by hhid and trip mode - original model
trips_org_sample_agg = trips_rsm.groupby(['hh_id', 'trip_mode'])['flag'].sum().reset_index()
trips_org_sample_agg = trips_org_sample_agg.rename(columns = {'flag': 'trips_org'})

#aggregating trips by hhid and trip mode - RSM
trips_rsm_agg = trips_rsm.groupby(['hh_id', 'trip_mode'])['flag'].sum().reset_index()
trips_rsm_agg = trips_rsm_agg.rename(columns = {'flag': 'trips_rsm'})

#combininging trips by hhid and trip mode
combined_trips = pd.merge(trips_org_sample_agg, trips_rsm_agg, on=['hh_id', 'trip_mode'], how = 'left') 

In [56]:
combined_trips

,hh_id,trip_mode,trips_org,trips_rsm
0,17,1,3,3
1,28,1,9,9
2,46,2,5,5
3,46,3,9,9
4,46,4,8,8
...,...,...,...,...
250666,1197099,2,1,1
250667,1197107,1,2,2
250668,1197109,2,1,1
250669,1197109,7,2,2


In [57]:
# aggregating by Home zone 
hh_rsm = pd.read_csv(os.path.join(output_dir_sample, 'households.csv'))
hh_rsm = hh_rsm[['hhid', 'mgra']]

In [60]:
# attach home zone id
combined_trips = pd.merge(combined_trips, hh_rsm, left_on = 'hh_id', right_on = 'hhid', how = 'left')

In [64]:
combined_trips_org = combined_trips.groupby('mgra')['trips_org'].sum().reset_index()
combined_trips_rsm = combined_trips.groupby('mgra')['trips_rsm'].sum().reset_index()

combined_trips = pd.merge(combined_trips_org, combined_trips_rsm, on = 'mgra', how = 'left')

In [66]:
combined_trips['perc_change'] = [x/y*100 for x,y in zip(combined_trips['trips_rsm'], combined_trips['trips_rsm'])]

In [67]:
combined_trips

,mgra,trips_org,trips_rsm,perc_change
0,1,253,253,100.0
1,2,221,221,100.0
2,4,41,41,100.0
3,6,53,53,100.0
4,8,21,21,100.0
...,...,...,...,...
1612,1996,3563,3563,100.0
1613,1997,1381,1381,100.0
1614,1998,1321,1321,100.0
1615,1999,862,862,100.0


In [69]:
combined_trips['perc_change'].describe()

count    1617.0
mean      100.0
std         0.0
min       100.0
25%       100.0
50%       100.0
75%       100.0
max       100.0
Name: perc_change, dtype: float64

In [ ]:
#INFO

In [5]:
ind_trips_full.columns

Index(['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
       'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra',
       'dest_mgra', 'parking_mgra', 'stop_period', 'trip_mode', 'av_avail',
       'trip_board_tap', 'trip_alight_tap', 'set', 'tour_mode', 'driver_pnum',
       'orig_escort_stoptype', 'orig_escortee_pnum', 'dest_escort_stoptype',
       'dest_escortee_pnum', 'valueOfTime', 'transponder_avail',
       'micro_walkMode', 'micro_trnAcc', 'micro_trnEgr', 'parkingCost'],
      dtype='object')

In [6]:
jnt_trips_full.columns

Index(['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose',
       'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
       'parking_mgra', 'stop_period', 'trip_mode', 'av_avail',
       'num_participants', 'trip_board_tap', 'trip_alight_tap', 'set',
       'tour_mode', 'valueOfTime', 'transponder_avail', 'parkingCost'],
      dtype='object')

In [9]:
ind_trips_rsm.columns

Index(['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
       'tour_purpose', 'orig_purpose', 'dest_purpose', 'orig_mgra',
       'dest_mgra', 'parking_mgra', 'stop_period', 'trip_mode', 'av_avail',
       'trip_board_tap', 'trip_alight_tap', 'set', 'tour_mode', 'driver_pnum',
       'orig_escort_stoptype', 'orig_escortee_pnum', 'dest_escort_stoptype',
       'dest_escortee_pnum', 'valueOfTime', 'transponder_avail',
       'micro_walkMode', 'micro_trnAcc', 'micro_trnEgr', 'parkingCost'],
      dtype='object')

In [8]:
jnt_trips_rsm.columns

Index(['hh_id', 'tour_id', 'stop_id', 'inbound', 'tour_purpose',
       'orig_purpose', 'dest_purpose', 'orig_mgra', 'dest_mgra',
       'parking_mgra', 'stop_period', 'trip_mode', 'av_avail',
       'num_participants', 'trip_board_tap', 'trip_alight_tap', 'set',
       'tour_mode', 'valueOfTime', 'transponder_avail', 'parkingCost'],
      dtype='object')

In [ ]:
tour0 = jot_tour[,.(hh_id,tour_id,tour_participants,tour_mode)]

In [15]:
jnt_trips_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518892 entries, 0 to 518891
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   hh_id              518892 non-null  int64  
 1   tour_id            518892 non-null  int64  
 2   stop_id            518892 non-null  int64  
 3   inbound            518892 non-null  int64  
 4   tour_purpose       518892 non-null  object 
 5   orig_purpose       518892 non-null  object 
 6   dest_purpose       518892 non-null  object 
 7   orig_mgra          518892 non-null  int64  
 8   dest_mgra          518892 non-null  int64  
 9   parking_mgra       518892 non-null  int64  
 10  stop_period        518892 non-null  int64  
 11  trip_mode          518892 non-null  int64  
 12  av_avail           518892 non-null  int64  
 13  num_participants   518892 non-null  int64  
 14  trip_board_tap     518892 non-null  int64  
 15  trip_alight_tap    518892 non-null  int64  
 16  se